# Unifed Content Recommendation

## By Rafeed Ullah

## Import Tools and Plugins

In [1]:
import pandas as pd
import numpy as np
import nltk
import re
import seaborn as sns

# Might use TensorFlows inbuild recommendation system
import tensorflow as tf

#SK Learn has some great Machine leaning Models
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise
import sklearn.metrics as metrics
from sklearn.metrics import silhouette_score
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from nltk.corpus import stopwords

# Some of my Favorite Data Visualization Tools
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

#NLP
from rake_nltk import Rake
#from rake_nltk import stopwords

from keybert import KeyBERT
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer

from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder as le
# ^ This didn't work

from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

from sklearn.metrics.pairwise import euclidean_distances

import itertools

from dateutil.parser import parse
from sklearn.feature_extraction.text import CountVectorizer



## Main DataFrame Setup

In [2]:
# Let's Take a peak at our dataframe
df= pd.read_csv("content.csv").drop(['duration'], axis = 1)

# Adding A Service Tag based on Content IDs
def serviceMapping(df):
    df['Streaming_Service'] = df['content_id'].str[0].map({'d': 'Disney Plus', 'p': 'Amazon Prime', 
                                                           'n': 'Netflix', 'h': 'Hulu'})
    return df

df = serviceMapping(df.copy())

#Axis = 1 means we are dropping it based on that first row rather than column
df.head(5)

,content_id,content_type,title,director,cast,country,date_added,release_year,rating,category,description,Streaming_Service
0,n1,Movie,Dick Johnson Is Dead,Kirsten Johnson,Not Available,United States,9/25/2021,2020.0,PG-13,Documentaries,"As her father nears the end of his life, filmm...",Netflix
1,n2,TV Show,Blood & Water,Not Available,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,9/24/2021,2021.0,TV-MA,"International, Dramas, Mystery","After crossing paths at a party, a Cape Town t...",Netflix
2,n3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",Not Available,9/24/2021,2021.0,TV-MA,"Crime, International, Action & Adventure",To protect his family from a powerful drug lor...,Netflix
3,n4,TV Show,Jailbirds New Orleans,Not Available,Not Available,Not Available,9/24/2021,2021.0,TV-MA,"Documentaries, Reality TV","Feuds, flirtations and toilet talk go down amo...",Netflix
4,n5,TV Show,Kota Factory,Not Available,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,9/24/2021,2021.0,TV-MA,"International, Romantic, Comedy",In a city of coaching centers known to train I...,Netflix


In [3]:
# What numerical metrics is the DF pulling
df.describe()

,release_year
count,20652.000000
mean,2011.254939
std,15.155224
min,1920.000000
25%,2011.000000
50%,2017.000000
75%,2019.000000
max,2021.000000


In [4]:
df.info()
df.corr()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21123 entries, 0 to 21122
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   content_id         21123 non-null  object 
 1   content_type       21123 non-null  object 
 2   title              21123 non-null  object 
 3   director           21123 non-null  object 
 4   cast               21123 non-null  object 
 5   country            21123 non-null  object 
 6   date_added         12202 non-null  object 
 7   release_year       20652 non-null  float64
 8   rating             21123 non-null  object 
 9   category           21123 non-null  object 
 10  description        18516 non-null  object 
 11  Streaming_Service  21123 non-null  object 
dtypes: float64(1), object(11)
memory usage: 1.9+ MB


,release_year
release_year,1.0


In [5]:
df.isna().sum() #Find Nulls

content_id              0
content_type            0
title                   0
director                0
cast                    0
country                 0
date_added           8921
release_year          471
rating                  0
category                0
description          2607
Streaming_Service       0
dtype: int64

### Clean-Up Data

In [6]:
# Create a new dataframe RecDF
RecDF = df[['category', 'release_year', 'title']]

# Filter out any rows with missing values
RecDF = RecDF.dropna()

# Convert the category and release_year columns to categorical data types
RecDF['category'] = RecDF['category'].astype('category')
RecDF['release_year'] = RecDF['release_year'].astype('category')

# Return the filtered dataframe
RecDF.head()

,category,release_year,title
0,Documentaries,2020.0,Dick Johnson Is Dead
1,"International, Dramas, Mystery",2021.0,Blood & Water
2,"Crime, International, Action & Adventure",2021.0,Ganglands
3,"Documentaries, Reality TV",2021.0,Jailbirds New Orleans
4,"International, Romantic, Comedy",2021.0,Kota Factory


Feature Engineering

In [7]:
# Define a function to one-hot encode the category
def one_hot_encode_category(df):
    categories = set()
    for category in df['category']:
        categories.update(category.split(", "))
    encoded_categories = pd.get_dummies(df['category'], columns=list(categories))
    return encoded_categories

In [8]:
'''
# Apply the function to the RecDF dataframe
RecDF_encoded_categories = one_hot_encode_category(RecDF.copy())

# Join the encoded categories back to the original RecDF dataframe
RecDF = pd.concat([RecDF, RecDF_encoded_categories], axis=1)

# Drop the original category column (optional)
RecDF = RecDF.drop('category', axis=1)

# Now, your RecDF dataframe will have separate columns for each category
print(RecDF)

      release_year                           title  Action  \
0           2020.0            Dick Johnson Is Dead       0   
1           2021.0                   Blood & Water       0   
2           2021.0                       Ganglands       0   
3           2021.0           Jailbirds New Orleans       0   
4           2021.0                    Kota Factory       0   
...            ...                             ...     ...   
21118       2009.0                   Samurai Harem       0   
21119       2015.0                   Scream Queens       0   
21120       1966.0  Star Trek: The Original Series       0   
21121       1959.0               The Twilight Zone       0   
21122       2009.0             Tokyo Magnitude 8.0       0   

       Action & Adventure  Action & Adventure, Anime  \
0                       0                          0   
1                       0                          0   
2                       0                          0   
3                       0      

In [9]:
from sentence_transformers import SentenceTransformer

# Optional: Download pre-trained model
model_name = "sentence-transformers/paraphrase-mpnet-base-v2"
model = SentenceTransformer(model_name)

In [10]:
# Lowercase and remove unnecessary characters
def preprocess_title(title):
    title = title.lower()
    title = re.sub(r"[^\w\s]", "", title)
    return title

# Apply preprocessing
titles = [preprocess_title(title) for title in RecDF['title']]

In [11]:
# Get sentence embeddings
title_embeddings = model.encode(titles)

In [12]:
np.save("title_embeddings.npy", title_embeddings) #We will need this later

In [13]:
# Add embeddings to RecDF
RecDF['title_embeddings'] = title_embeddings[:, 0]

In [14]:
# Troubleshoot

print(f"RecDF shape: {RecDF.shape}")
print(f"title_embeddings shape: {title_embeddings.shape}")

RecDF shape: (20652, 1512)
title_embeddings shape: (20652, 768)


In [15]:
RecDF.head()

,release_year,title,Action,Action & Adventure,"Action & Adventure, Anime","Action & Adventure, Anime, Classic Movies","Action & Adventure, Anime, Family","Action & Adventure, Anime, Horror","Action & Adventure, Anime, International","Action & Adventure, Anime, Sci-Fi & Fantasy",...,"Suspense, Western",TV Shows,Talk Show,Talk Show and Variety,Thriller,Unscripted,Variety,Western,Young Adult Audience,title_embeddings
0,2020.0,Dick Johnson Is Dead,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.098891
1,2021.0,Blood & Water,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-0.093786
2,2021.0,Ganglands,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-0.086835
3,2021.0,Jailbirds New Orleans,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-0.080067
4,2021.0,Kota Factory,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-0.099033


In [16]:
# Choose a suitable distance metric (e.g., cosine similarity for embeddings)
# distance_metric = metrics.pairwise.cosine_similarity

In [17]:
print(f"Title embeddings shape: {title_embeddings.shape}")
print(f"Release year shape: {RecDF['release_year'].shape}")
print(f"Encoded categories shape: {RecDF.iloc[:, 15:].to_numpy().shape}")

Title embeddings shape: (20652, 768)
Release year shape: (20652,)
Encoded categories shape: (20652, 1497)


In [18]:
print(f"Title embeddings shape: {title_embeddings.shape}")
print(f"Release year shape: {RecDF['release_year'].shape}")
print(f"Encoded categories shape: {RecDF.iloc[:, 15:].to_numpy().shape}")

Title embeddings shape: (20652, 768)
Release year shape: (20652,)
Encoded categories shape: (20652, 1497)


In [19]:
'''
# Combine features into a single array
features = np.concatenate([title_embeddings, RecDF[['release_year']], RecDF.iloc[:, 3:-1].to_numpy()], axis=1)

In [20]:
'''
# Define the number of clusters (adjust based on your needs)
n_clusters = 11

# Change the linkage method to 'average'
AHCmodel = AgglomerativeClustering(n_clusters=11, linkage='average')

# Fit the model with the cosine similarity metric
AHCmodel.fit(features)

AgglomerativeClustering(linkage='average', n_clusters=11)

In [21]:
'''
# Assign each data point to a cluster
cluster_labels = AHCmodel.labels_

# Add cluster labels to the RecDF dataframe
RecDF['cluster_label'] = cluster_labels

In [22]:
#Cluster mapping, candidate generation, and ranking

def map_clusters(titles, AHCmodel):
    from sklearn.metrics.pairwise import cosine_similarity

    # Get title embeddings
    title_embeddings = np.load("title_embeddings.npy")
    title_embeddings = title_embeddings.reshape(-1, 1) if title_embeddings.ndim == 1 else title_embeddings
    # Calculate cosine similarities between titles
    similarities = cosine_similarity(title_embeddings)

    # Ensure similarities are a 2D array
    if similarities.ndim != 2:
        raise ValueError("Similarities matrix should be a 2D array")
    
    # Create and fit a temporary Agglomerative Clustering model
    temp_model = AgglomerativeClustering(n_clusters=11, affinity='precomputed', linkage='average')
    temp_model.fit(similarities)

    # Predict cluster labels for all titles
    cluster_labels = temp_model.labels_

    # Create a dictionary mapping titles to their cluster labels
    cluster_dict = {}
    for i, title in enumerate(titles):
        cluster_dict[title] = cluster_labels[i]

    return cluster_dict

In [23]:
def rank_candidates(title, candidates, title_embeddings):
    # Check if title embeddings are 1D
    if title_embeddings.ndim == 1:
        # Reshape the title embeddings to 2D
        title_embeddings = title_embeddings.reshape(-1, 1)

    # Validate candidates and filter out invalid ones
    valid_candidates = [c for c in candidates if c in title_embeddings]

    if not valid_candidates:
        raise ValueError("No valid titles found in candidates list")

    # Calculate the average embedding of valid candidates
    average_embedding = np.mean([title_embeddings[c] for c in valid_candidates], axis=0)

    try:
        # Generate a title based on the average embedding
        generated_title = generate_title(average_embedding)
    except Exception as e:
        # Handle any errors during title generation
        raise ValueError(f"Failed to generate title: {e}")

    # Return the generated title as the output of the function
    return generated_title

In [24]:
candidates = ["Meet the Robinsons", "Minutemen", "Avatar: The Last Airbender"]
# title_embeddings = np.load("title_embeddings.npy")

In [25]:
print(candidates)

['Meet the Robinsons', 'Minutemen', 'Avatar: The Last Airbender']


In [26]:
'''
# Get the title embeddings
title_embeddings = np.load("title_embeddings.npy")

# Generate a single title output
try:
    generated_title = rank_candidates("My Title", candidates, title_embeddings)
except ValueError as e:
    print(f"Error generating title: {e}")
else:
    print(f"Generated title: {generated_title}")

Error generating title: No valid titles found in candidates list


In [27]:
valid_candidates = [c for c in candidates if c in title_embeddings]

average_embedding = np.mean([title_embeddings[c] for c in valid_candidates], axis=0)

C:\Users\RRUll\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\RRUll\anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [28]:
'''
for title in title_embeddings:
    embedding = title_embeddings[title]
    if np.isnan(embedding).any() or not np.isfinite(embedding).all():
        print(f"Invalid value found in embedding for title: {title}")
'''

'\nfor title in title_embeddings:\n    embedding = title_embeddings[title]\n    if np.isnan(embedding).any() or not np.isfinite(embedding).all():\n        print(f"Invalid value found in embedding for title: {title}")\n'

Let's Try Content-based

In [29]:
CBDF = df[['title', 'director', 'cast', 'category', 'release_year', 'country']]
# Filter out any rows with missing values
CBDF = CBDF.dropna()

In [30]:
CBDF.head()

,title,director,cast,category,release_year,country
0,Dick Johnson Is Dead,Kirsten Johnson,Not Available,Documentaries,2020.0,United States
1,Blood & Water,Not Available,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...","International, Dramas, Mystery",2021.0,South Africa
2,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...","Crime, International, Action & Adventure",2021.0,Not Available
3,Jailbirds New Orleans,Not Available,Not Available,"Documentaries, Reality TV",2021.0,Not Available
4,Kota Factory,Not Available,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...","International, Romantic, Comedy",2021.0,India


In [31]:
actors = []

for i in CBDF['cast']:
    actor = re.split(r', \s*', i)
    actors.append(actor)
    
flat_list = []
for sublist in actors:
    for item in sublist:
        flat_list.append(item)
        
actors_list = sorted(set(flat_list))

binary_actors = [[0] * 0 for i in range(len(set(flat_list)))]

for i in CBDF['cast']:
    k = 0
    for j in actors_list:
        if j in i:
            binary_actors[k].append(1.0)
        else:
            binary_actors[k].append(0.0)
        k+=1
        
binary_actors = pd.DataFrame(binary_actors).transpose()

In [32]:
########################################################################

directors = []

for i in CBDF['director']:
    if pd.notna(i):
        director = re.split(r', \s*', i)
        directors.append(director)
    
flat_list2 = []
for sublist in directors:
    for item in sublist:
        flat_list2.append(item)
        
directors_list = sorted(set(flat_list2))

binary_directors = [[0] * 0 for i in range(len(set(flat_list2)))]
for i in CBDF['director']:  
    k = 0
    for j in directors_list:
        if pd.isna(i):
            binary_directors[k].append(0.0)
        elif j in i:
            binary_directors[k].append(1.0)
        else:
            binary_directors[k].append(0.0)
        k+=1
        
binary_directors = pd.DataFrame(binary_directors).transpose()

In [33]:
genres = []

for i in CBDF['category']:
    genre = re.split(r', \s*', i)
    genres.append(genre)
    
flat_list4 = []
for sublist in genres:
    for item in sublist:
        flat_list4.append(item)
        
genres_list = sorted(set(flat_list4))

binary_genres = [[0] * 0 for i in range(len(set(flat_list4)))]

for i in CBDF['category']:
    k = 0
    for j in genres_list:
        if j in i:
            binary_genres[k].append(1.0)
        else:
            binary_genres[k].append(0.0)
        k+=1

binary_genres = pd.DataFrame(binary_genres).transpose()

In [67]:
CBDF.head()

,title,director,cast,category,release_year,country,cos_sim
0,Dick Johnson Is Dead,Kirsten Johnson,Not Available,Documentaries,2020.0,United States,0.484799
1,Blood & Water,Not Available,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...","International, Dramas, Mystery",2021.0,South Africa,0.756602
2,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...","Crime, International, Action & Adventure",2021.0,Not Available,0.498123
3,Jailbirds New Orleans,Not Available,Not Available,"Documentaries, Reality TV",2021.0,Not Available,0.669150
4,Kota Factory,Not Available,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...","International, Romantic, Comedy",2021.0,India,0.708869


In [34]:
binary = pd.concat([binary_actors, binary_directors, binary_genres], axis=1,ignore_index=True)
binary

,0,1,2,3,4,5,6,7,8,9,...,69181,69182,69183,69184,69185,69186,69187,69188,69189,69190
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20647,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20648,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20649,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20650,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [59]:
def recommender(search, totalResults):
    cs_list = []
    binary_list = []
    #Check for user input
    if search in CBDF['title'].values:
        
        #Extract Binary Representation and Convert it to an array
        idx = CBDF[CBDF['title'] == search].index.item()
        for i in binary.iloc[idx]:
            binary_list.append(i)
        
        point1 = np.array(binary_list).reshape(1, -1)
        point1 = [val for sublist in point1 for val in sublist]    
        
        #Calculate Cosine Similarity for all titles
        for j in range(len(CBDF)):
            binary_list2 = []
            
            for k in binary.iloc[j]:
                binary_list2.append(k)
            point2 = np.array(binary_list2).reshape(1, -1)
            point2 = [val for sublist in point2 for val in sublist]
            dot_product = np.dot(point1, point2)
            norm_1 = np.linalg.norm(point1)
            norm_2 = np.linalg.norm(point2)
            cos_sim = dot_product / (norm_1 * norm_2)
            cs_list.append(cos_sim)
        
        CBDF_copy = CBDF.copy()
        CBDF_copy['cos_sim'] = cs_list
        
        results = CBDF_copy.sort_values('cos_sim', ascending=False)
        results = results[results['title'] != search]
        
        merge_columns = ['title', 'director', 'cast', 'category', 'release_year', 'country']
        # Merge the dataframes based on the dupe column
        # totalResults = totalResults.merge(results, on=merge_columns, how='inner')

        # Add the corresponding values of the 'cos_sim' column
        # Add the cosine similarity values from the results dataframe
        totalResults['cos_sim'] = totalResults['cos_sim'].add(results['cos_sim'], fill_value=0)
        
        top_results = results.head(5)
        return(totalResults)
    
    else:
        return("Title not in dataset. Please check spelling.")
        return(totalResults)

In [69]:

def recommend_titles1(search1):
    cs_list = []
    binary_list = []
    #Check for user input
    if search1 in CBDF['title'].values:
        
        #Extract Binary Representation and Convert it to an array
        idx = CBDF[CBDF['title'] == search1].index.item()
        for i in binary.iloc[idx]:
            binary_list.append(i)
        
        point1 = np.array(binary_list).reshape(1, -1)
        point1 = [val for sublist in point1 for val in sublist]    
        
        #Calculate Cosine Similarity for all titles
        for j in range(len(CBDF)):
            binary_list2 = []
            
            for k in binary.iloc[j]:
                binary_list2.append(k)
            point2 = np.array(binary_list2).reshape(1, -1)
            point2 = [val for sublist in point2 for val in sublist]
            dot_product = np.dot(point1, point2)
            norm_1 = np.linalg.norm(point1)
            norm_2 = np.linalg.norm(point2)
            cos_sim = dot_product / (norm_1 * norm_2)
            cs_list.append(cos_sim)
        
        CBDF_copy = CBDF.copy()
        CBDF_copy['cos_sim'] = cs_list
        
        results = CBDF_copy.sort_values('cos_sim', ascending=False)
        results = results[results['title'] != search1]    
        top_results = results.head(5)
        return(top_results)
    
    else:
        return("Title not in dataset. Please check spelling.")
        return(top_results)


In [70]:

def recommend_titles2(search2):
    cs_list = []
    binary_list = []
    #Check for user input
    if search2 in CBDF['title'].values:
        
        #Extract Binary Representation and Convert it to an array
        idx = CBDF[CBDF['title'] == search2].index.item()
        for i in binary.iloc[idx]:
            binary_list.append(i)
        
        point1 = np.array(binary_list).reshape(1, -1)
        point1 = [val for sublist in point1 for val in sublist]    
        
        #Calculate Cosine Similarity for all titles
        for j in range(len(CBDF)):
            binary_list2 = []
            
            for k in binary.iloc[j]:
                binary_list2.append(k)
            point2 = np.array(binary_list2).reshape(1, -1)
            point2 = [val for sublist in point2 for val in sublist]
            dot_product = np.dot(point1, point2)
            norm_1 = np.linalg.norm(point1)
            norm_2 = np.linalg.norm(point2)
            cos_sim = dot_product / (norm_1 * norm_2)
            cs_list.append(cos_sim)
        
        CBDF_copy = CBDF.copy()
        CBDF_copy['cos_sim'] = cs_list
        
        results = CBDF_copy.sort_values('cos_sim', ascending=False)
        results = results[results['title'] != search2]    
        top_results = results.head(5)
        return(top_results)
    
    else:
        return("Title not in dataset. Please check spelling.")
        return(top_results)


In [38]:
'''
def recommend_titles3(search3):
    cs_list = []
    binary_list = []
    #Check for user input
    if search3 in CBDF['title'].values:
        
        #Extract Binary Representation and Convert it to an array
        idx = CBDF[CBDF['title'] == search3].index.item()
        for i in binary.iloc[idx]:
            binary_list.append(i)
        
        point1 = np.array(binary_list).reshape(1, -1)
        point1 = [val for sublist in point1 for val in sublist]    
        
        #Calculate Cosine Similarity for all titles
        for j in range(len(CBDF)):
            binary_list2 = []
            
            for k in binary.iloc[j]:
                binary_list2.append(k)
            point2 = np.array(binary_list2).reshape(1, -1)
            point2 = [val for sublist in point2 for val in sublist]
            dot_product = np.dot(point1, point2)
            norm_1 = np.linalg.norm(point1)
            norm_2 = np.linalg.norm(point2)
            cos_sim = dot_product / (norm_1 * norm_2)
            cs_list.append(cos_sim)
        
        CBDF_copy = CBDF.copy()
        CBDF_copy['cos_sim'] = cs_list
        
        results = CBDF_copy.sort_values('cos_sim', ascending=False)
        results = results[results['title'] != search3]
        top_results = results.head(5)
        return(top_results)
    
    else:
        return("Title not in dataset. Please check spelling.")
        return(top_results)
'''

'\ndef recommend_titles3(search3):\n    cs_list = []\n    binary_list = []\n    #Check for user input\n    if search3 in CBDF[\'title\'].values:\n        \n        #Extract Binary Representation and Convert it to an array\n        idx = CBDF[CBDF[\'title\'] == search3].index.item()\n        for i in binary.iloc[idx]:\n            binary_list.append(i)\n        \n        point1 = np.array(binary_list).reshape(1, -1)\n        point1 = [val for sublist in point1 for val in sublist]    \n        \n        #Calculate Cosine Similarity for all titles\n        for j in range(len(CBDF)):\n            binary_list2 = []\n            \n            for k in binary.iloc[j]:\n                binary_list2.append(k)\n            point2 = np.array(binary_list2).reshape(1, -1)\n            point2 = [val for sublist in point2 for val in sublist]\n            dot_product = np.dot(point1, point2)\n            norm_1 = np.linalg.norm(point1)\n            norm_2 = np.linalg.norm(point2)\n            cos_sim 

In [71]:
recommend_titles1('Nevertheless,')

,title,director,cast,category,release_year,country,cos_sim
6083,Club Friday To Be Continued - The Promise,Not Available,"Jirayu La-ongmanee, Focus Jirakul","International, Romantic, Dramas",2016.0,Not Available,0.673575
5879,"Berlin, I Love You",Not Available,Not Available,"Dramas, International, Romantic",2019.0,Germany,0.612372
2157,Seven (Tamil),Not Available,Not Available,"Dramas, International, Romantic",2019.0,India,0.612372
7465,Sotus The Series,Not Available,"Sangpotirat Perawat, Ruangroj Prachaya","International, Romantic, Dramas",2016.0,Thailand,0.612372
2158,Seven (Telugu),Not Available,Not Available,"Dramas, International, Romantic",2019.0,Not Available,0.612372


In [72]:
recommend_titles2('Itaewon Class')

,title,director,cast,category,release_year,country,cos_sim
7571,The Adjusters,Not Available,"Rynn Lim, Chris Tong, Aric Ho, Chan Fong","International, Dramas",2010.0,Not Available,0.516398
968,Ajaibnya Cinta,Not Available,Not Available,"International, Dramas",2015.0,Not Available,0.507093
62,Stories by Rabindranath Tagore,Not Available,Not Available,"International, Dramas",2015.0,India,0.507093
1186,Zindagi in Short,Not Available,Not Available,"International, Dramas",2020.0,India,0.507093
7739,The Lady in Dignity,Not Available,"Hee-seon Kim, Seon-a Kim, Sang-hun Jeong, Tae-...","International, Korean, Dramas",2017.0,South Korea,0.496139


In [41]:
# recommend_titles3('Ratatouille')
# Fail but Understandable as Ratatouille has no release year 
# in the dataset at time of running this

# recommend_titles3('Bridgerton')

In [42]:
#combine and add cos_sim of each resukt then sort by highest

CBDF.loc[CBDF['title'] == 'Itaewon Class']

,title,director,cast,category,release_year,country
2733,Itaewon Class,Not Available,"Park Seo-joon, Kim Da-mi, Yu Jae-myeong, Kwon ...","International, Korean, Dramas",2020.0,South Korea


In [43]:
CBDF.loc[CBDF['title'] == 'Nevertheless,']

,title,director,cast,category,release_year,country
606,"Nevertheless,",Not Available,"Song Kang, Han So-hee, Chae Jong-hyeop, Lee Yu...","International, Romantic, Dramas",2021.0,Not Available


In [44]:
CBDF.loc[CBDF['title'] == 'Bridgerton']

,title,director,cast,category,release_year,country
1388,Bridgerton,Not Available,"Adjoa Andoh, Julie Andrews, Lorraine Ashbourne...","Romantic, Dramas",2020.0,United States


In [60]:
totalResults = CBDF
# Create the cos_sim column
totalResults['cos_sim'] = 0

In [61]:
search_terms = ['Nevertheless,', 'Bridgerton', 'Itaewon Class']
totalResults.head()

,title,director,cast,category,release_year,country,cos_sim
0,Dick Johnson Is Dead,Kirsten Johnson,Not Available,Documentaries,2020.0,United States,0
1,Blood & Water,Not Available,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...","International, Dramas, Mystery",2021.0,South Africa,0
2,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...","Crime, International, Action & Adventure",2021.0,Not Available,0
3,Jailbirds New Orleans,Not Available,Not Available,"Documentaries, Reality TV",2021.0,Not Available,0
4,Kota Factory,Not Available,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...","International, Romantic, Comedy",2021.0,India,0


In [62]:
i=0
search_term = search_terms[i]


for search_term in search_terms:
    recommender(search_term, totalResults)
    i+=1
    search_term = search_terms[i-1]

In [63]:
totalResults.head(5)

,title,director,cast,category,release_year,country,cos_sim
0,Dick Johnson Is Dead,Kirsten Johnson,Not Available,Documentaries,2020.0,United States,0.484799
1,Blood & Water,Not Available,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...","International, Dramas, Mystery",2021.0,South Africa,0.756602
2,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...","Crime, International, Action & Adventure",2021.0,Not Available,0.498123
3,Jailbirds New Orleans,Not Available,Not Available,"Documentaries, Reality TV",2021.0,Not Available,0.669150
4,Kota Factory,Not Available,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...","International, Romantic, Comedy",2021.0,India,0.708869


In [64]:
totalResults = totalResults.sort_values('cos_sim', ascending=False)
# totalResults.head(5)

# First Try had a sorting error, not cl

In [65]:
totalResults.head(5)

,title,director,cast,category,release_year,country,cos_sim
6083,Club Friday To Be Continued - The Promise,Not Available,"Jirayu La-ongmanee, Focus Jirakul","International, Romantic, Dramas",2016.0,Not Available,1.468944
2243,The Light of My Eyes,Not Available,Not Available,"Dramas, International, Romantic",2010.0,Not Available,1.455989
5879,"Berlin, I Love You",Not Available,Not Available,"Dramas, International, Romantic",2019.0,Germany,1.455989
2158,Seven (Telugu),Not Available,Not Available,"Dramas, International, Romantic",2019.0,Not Available,1.455989
2157,Seven (Tamil),Not Available,Not Available,"Dramas, International, Romantic",2019.0,India,1.455989


In [2]:
# Attempt 2
totalResults = CBDF
# Create the cos_sim column
totalResults['cos_sim'] = 0
search_terms = ['Big Hero 6', 'Peaky Blinders', 'High School Musical 3: Senior Year']

NameError: name 'CBDF' is not defined

In [1]:
i=0
search_term = search_terms[i]


for search_term in search_terms:
    recommender(search_term, totalResults)
    i+=1
    search_term = search_terms[i-1]
    
totalResults = totalResults.sort_values('cos_sim', ascending=False)

NameError: name 'search_terms' is not defined

In [ ]:
totalResults.head(5)

Keyword Extraction and predictions (Attempt 2)

In [ ]:
rake = Rake()

# Extract the keywords from each title
keywords = []
for title in df['title']:
    keywords.append(rake.extract_keywords(title))

In [ ]:
# Flatten the list of keywords
flat_keywords = []
for keyword_list in keywords:
    flat_keywords += keyword_list

# Create a new DataFrame with the titles and keywords
KeywordDF = pd.DataFrame({'title': df['title'], 'keywords': flat_keywords})

# Remove duplicate rows
KeywordDF = KeywordDF.drop_duplicates()

In [ ]:
KeywordDF.head(10)